In [1]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset, load_dataset_builder

ds = load_dataset("aashsach/multiconer2", "en")



Found cached dataset multiconer2 (/users/jmperez/.cache/huggingface/datasets/aashsach___multiconer2/en/1.0.0/f8781e365f606105430f8dae7fad131df6b2e8cd35c0973165579f0ddc8a1bbc)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
ds = ds.map(lambda ex: {"text": " ".join(ex["tokens"])})

Loading cached processed dataset at /users/jmperez/.cache/huggingface/datasets/aashsach___multiconer2/en/1.0.0/f8781e365f606105430f8dae7fad131df6b2e8cd35c0973165579f0ddc8a1bbc/cache-65691de3e43a9f42.arrow
Loading cached processed dataset at /users/jmperez/.cache/huggingface/datasets/aashsach___multiconer2/en/1.0.0/f8781e365f606105430f8dae7fad131df6b2e8cd35c0973165579f0ddc8a1bbc/cache-7354ddfb71e081c2.arrow


In [4]:
import requests
from pysentimiento.config import config

# URL for the Google Knowledge Graph Search API
GOOGLE_KG_URL = 'https://kgsearch.googleapis.com/v1/entities:search'

# URL for retrieving wikidata id
WIKIDATA_URL_TEMPLATE = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles={}&format=json"

API_KEY = config['GOOGLE']['API_KEY']

def kg_search(query, api_key=API_KEY, limit=10, indent=True, **kwargs):
    """
    Performs a Knowledge Graph Search query.

    Args:

        query (str): The query string.

        params (dict): The parameters to pass to the API.

    Returns:

        dict: The JSON response from the API.
    """

    # Remove accents from query
    query = query.encode('ascii', 'ignore').decode('ascii')
    # Merge params and kwargs
    params = {
        'query': query,
        'key': api_key,
        'limit': limit,
        'indent': indent,
    }
    params.update(kwargs)

    req = requests.get(GOOGLE_KG_URL, params=params)
    jsn = req.json()

    return jsn


In [5]:
id2label = ds["train"].features["ner_tags"].feature.names

id2label = {i: label for i, label in enumerate(id2label)}
label2id = {label: i for i, label in id2label.items()}

In [6]:
from pysentimiento.ner import tokenize_and_align_labels, compute_metrics
from pysentimiento.training import train_and_eval
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
from pysentimiento.tuning import get_training_arguments

base_model = "roberta-large"


training_args = get_training_arguments(
    base_model, task_name="multiconer", lang="en",
    metric_for_best_model="eval/micro_f1", use_defaults_if_not_tuned=True
)

training_args.num_train_epochs = 5


/users/jmperez/projects/pysentimiento/pysentimiento/ner.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [7]:
import numpy as np
from seqeval.metrics import f1_score
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(eval_preds, id2label):
    """
    Compute metrics for NER
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100]
                   for label in labels]

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # I also calculate mention metrics
    # That is, we ignore the I- and B- tags and only consider O and MENTION
    def label_to_mention(label):
        if label == "O":
            return "O"
        elif label.startswith("B-"):
            return "B-MENTION"
        else:
            return "I-MENTION"


    mention_labels = [
        [label_to_mention(l) for l in labels]
        for labels in true_labels
    ]

    mention_predictions = [
        [label_to_mention(p) for p in labels]
        for labels in true_predictions
    ]
    
    all_metrics = metric.compute(
        predictions=true_predictions, references=true_labels)
    
    mention_metrics = metric.compute(
        predictions=mention_predictions, references=mention_labels)

    ret = {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "mention_precision": mention_metrics["overall_precision"],
        "mention_recall": mention_metrics["overall_recall"],
        "mention_f1": mention_metrics["overall_f1"],
        "macro_f1": f1_score(true_labels, true_predictions, average="macro"),
        "micro_f1": f1_score(true_labels, true_predictions, average="micro"),
        "accuracy": all_metrics["overall_accuracy"],
    }

    for k, v in all_metrics.items():
        if not k.startswith("overall"):
            ret[k + "_f1"] = v["f1"]
            ret[k + "_precision"] = v["precision"]
            ret[k + "_recall"] = v["recall"]

    return ret



In [8]:


ds["dev"] = ds["validation"]
ds["test"] = ds["validation"]


train_and_eval(
    base_model, dataset=ds,
    id2label=id2label, lang="en", 
    training_args=training_args,
    # Custom stuff for this thing to work
    tokenize_fun=tokenize_and_align_labels,
    auto_class=AutoModelForTokenClassification,
    data_collator_class=DataCollatorForTokenClassification,
    metrics_fun=lambda x: compute_metrics(x, id2label),
    metric_for_best_model="micro_f1"
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: text, ner_tags, word_ids, ner_macro_tags, tokens, id. If text, ner_tags, word_ids, ner_macro_tags, tokens, id are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/users/jmperez/projects/pysentimiento/.venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16778
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2625
  Number of trainable parameters = 354381891
Automatic

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
